In [38]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os
from dotenv import load_dotenv
import yaml

load_dotenv(".env")

with open("agents/k8s_expert_agent/k8s_agent.yaml", "r") as file:
    settings = yaml.safe_load(file)

k8s_expert_agent_id = settings["agent_id"]

with open("agents/save_task_agent/k8s_agent.yaml", "r") as file:
    settings = yaml.safe_load(file)
save_k8s_task_agent_id = settings["agent_id"]

In [39]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    model=os.getenv("MODE_NAME"),
    api_version=os.getenv("API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
)

In [40]:

from autogen_core.models import UserMessage
result = await az_model_client.create(
    [UserMessage(content="What is the capital of France?", source="user")]
)
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None


In [41]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [42]:
async def generate_k8s_task_agent(content: str) -> str:        
    agent = project_client.agents.get_agent(k8s_expert_agent_id)

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=content,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg =  messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")
    return last_msg

In [43]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    tools=[generate_k8s_task_agent],
    system_message="""
        You are a kubernetes task writer, please help me write a kubernetes task based on bing search content."
    """
)

In [44]:
async def save_task_agent(content: str) -> str:
    agent = project_client.agents.get_agent(save_k8s_task_agent_id)

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""
        
                    You are my Python programming assistant. Generate code,save """
        + content
        + """    
                    and execute it according to the following requirements

                    1. Save task content to task-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id, assistant_id=agent.id
    )
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")

    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(
            file_id=file_path_annotation.file_path.file_id,
            file_name=file_name,
            target_dir="./task",
        )

        # project_client.close()

    return "Saved"

In [45]:
save_k8s_task_agent = AssistantAgent(
    name="save_k8s_task_agent",
    model_client=az_model_client,
    tools=[save_task_agent],
    system_message="""
        Save kubernetes task. Respond with 'Saved' to when your task are saved.
    """
)

In [46]:
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

bing_search_agent = MultimodalWebSurfer(
        name="web_surfer",
        model_client=az_model_client
    )

In [47]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [48]:
reflection_team = RoundRobinGroupChat(
    [bing_search_agent, write_agent, save_k8s_task_agent],
    termination_condition=termination,
)

In [49]:
await Console(
    reflection_team.run_stream(
        task="""
I am instructor to create kubernetes exercise task.
Help me write question and answer for the exercise task
For example:
    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.
Answer:
    First use kubectl:
                               
    kubectl run nginx --image=nginx:latest --port=80
                               
    Then use YAML file:
                               
    apiVersion: v1
    kind: Pod
    metadata:
      name: nginx
    spec:                               
        containers:
        - name: nginx
          image: nginx:latest
          ports:
          - containerPort: 80
    
    
 Write the new question and answer.                            

    """
    )
)  # Stream the messages to the console.

---------- user ----------

I am instructor to create kubernetes exercise task.
Help me write question and answer for the exercise task
For example:
    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.
Answer:
    First use kubectl:
                               
    kubectl run nginx --image=nginx:latest --port=80
                               
    Then use YAML file:
                               
    apiVersion: v1
    kind: Pod
    metadata:
      name: nginx
    spec:                               
        containers:
        - name: nginx
          image: nginx:latest
          ports:
          - containerPort: 80
    
    
 Write the new question and answer.                            

    
---------- web_surfer ----------
**Question:**
Create a deployment named 'my-app' with 3 replicas using the image 'my-app:1.0'. The deployment should expose port 8080.

**Answer:**
First, use kubectl:

```bash
kubectl create deployment my-app --image=my-app:1

/workspaces/k8s-game-generator/venv/lib/python3.13/site-packages/msal/token_cache.py:164: DeprecationWarning: Use list(search(...)) instead to explicitly get a list.
  warnings.warn(


Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_HweK5HVGGmA1nDw5qlv7U0q8', 'object': 'thread.message', 'created_at': 1741597158, 'assistant_id': 'asst_ujCbHwxZvxUZrzPw8ku7RqXP', 'thread_id': 'thread_06DGqLkeUGZT8TnuId5aPwOn', 'run_id': 'run_gVMab78k5QSC7soc85m0v9lY', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "To create a service named 'my-app-service' that exposes the deployment 'my-app' on port 8080 and maps it to a NodePort, you can use the following YAML configuration:\n\n```yaml\napiVersion: v1\nkind: Service\nmetadata:\n  name: my-app-service\nspec:\n  type: NodePort\n  selector:\n    app: my-app\n  ports:\n    - port: 8080\n      targetPort: 8080\n      nodePort: 30001  # Optional: Specify a nodePort or let Kubernetes assign one\n```\n\n### Explanation:\n- **apiVersion**: Specifies the API version of the Kubernetes object.\n- **kind**: Defines the type of object, in this case, a Service.\n- **metadata**: Co

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="\nI am instructor to create kubernetes exercise task.\nHelp me write question and answer for the exercise task\nFor example:\n    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.\nAnswer:\n    First use kubectl:\n                               \n    kubectl run nginx --image=nginx:latest --port=80\n                               \n    Then use YAML file:\n                               \n    apiVersion: v1\n    kind: Pod\n    metadata:\n      name: nginx\n    spec:                               \n        containers:\n        - name: nginx\n          image: nginx:latest\n          ports:\n          - containerPort: 80\n    \n    \n Write the new question and answer.                            \n\n    ", type='TextMessage'), TextMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=38651, completion_tokens=178), content="**Question:**\nCreate a deployment named 'my-app